In [1]:
import sys
sys.path.append("../")
sys.path.append("../../dcdi")

In [8]:
import numpy as np
import pandas as pd
import torch
import wandb
import matplotlib.pyplot as plt

from models import SDCI
from utils import create_intervention_dataset, subset_interventions, train_val_split, compute_metrics, set_random_seed_all
from third_party.utils.metrics import shd_metric

In [3]:
from dcdi.data import DataManagerFile

In [4]:
DataManagerFile.__init__

<function dcdi.data.DataManagerFile.__init__(self, file_path, i_dataset, train_samples=0.8, test_samples=None, train=True, normalize=False, mean=None, std=None, random_seed=42, intervention=False, intervention_knowledge='known', dcd=False, regimes_to_ignore=None)>

In [6]:
# dataset_name = "data_p10_e10_n10000_linear_struct"
# dataset_i = 1

# dmf = DataManagerFile(
#     f"../../dcdi/data/perfect/{dataset_name}",
#     dataset_i,
#     train_samples=1.0, # grab whole dataset
#     train=True,
#     normalize=True,
#     intervention=True,
#     intervention_knowledge="known"
# )
# X = dmf.dataset
# masks = dmf.convert_masks(np.arange(X.shape[0]))
# n_regimes = (1 - masks).sum(axis=1)
# B_true = dmf.adjacency.numpy()

In [7]:
# dataset = torch.utils.data.TensorDataset(X, masks, n_regimes)
# np.random.seed(2)
# train_dataset, val_dataset = train_val_split(dataset, flavor="random", val_fraction=0.2)

In [8]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(f"Using {device}")

Using cuda:0


In [5]:
def load_dcdi_data(dataset_name, dataset_i):
    dmf = DataManagerFile(
        f"../../dcdi/data/perfect/{dataset_name}",
        dataset_i,
        train_samples=1.0, # grab whole dataset
        train=True,
        normalize=True,
        intervention=True,
        intervention_knowledge="known"
    )
    X = dmf.dataset
    masks = dmf.convert_masks(np.arange(X.shape[0]))
    n_regimes = (1 - masks).sum(axis=1)
    B_true = dmf.adjacency.numpy()
    dataset = torch.utils.data.TensorDataset(X, masks, n_regimes)
    return dataset, B_true

In [9]:
def run_sdci_tuned(dataset, seed=42, device=None):
    set_random_seed_all(seed)
    train_dataset, val_dataset = train_val_split(dataset, flavor="random", val_fraction=0.2)
    stage1_kwargs = {
        "n_epochs": 2000,
        "alpha": 1e-2,
        "beta": 2e-4
    }
    stage2_kwargs = {
        "learning_rate": 2e-3,
        "n_epochs": 2000,
        "alpha": 5e-5,
        "beta": 5e-5, 
        "max_gamma": 200
    }
    model = SDCI(model_variance_flavor="nn")
    model.train(
        train_dataset,
        val_dataset=val_dataset,
        B_true=B_true,
        device=device,
        verbose=True,
        log_wandb=True,
        wandb_project=f"{dataset_name}",
        wandb_name=f"SDCI_{dataset_i}",
        stage1_kwargs=stage1_kwargs,
        stage2_kwargs=stage2_kwargs,
    )
    y_test = B_true
    y_score = model.get_adjacency_matrix(threshold = True)
    return compute_metrics(y_score, y_test)

In [ ]:
dataset_names = """
data_p10_e10_n10000_linear_struct
data_p20_e20_n10000_nnadd_struct
data_p10_e10_n10000_nnadd_struct
data_p20_e20_n10000_nn_struct
data_p10_e10_n10000_nn_struct
data_p20_e80_n10000_linear_struct
data_p10_e40_n10000_linear_struct
data_p20_e80_n10000_nnadd_struct
data_p10_e40_n10000_nnadd_struct
data_p20_e80_n10000_nn_struct
data_p10_e40_n10000_nn_struct
data_p20_e20_n10000_linear_struct
""".split()
dataset_is = np.arange(1, 11)

seed = 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

for dataset_name in dataset_names: 
    results_df_rows = []
    for dataset_i in dataset_is:
        dataset, B_true = load_dcdi_data(dataset_name, dataset_i)
    
        np.random.seed(seed)
        res_metrics = run_sdci_tuned(dataset, seed=seed, device=device)
        res_metrics["dataset_i"] = dataset_i
        results_df_rows.append(res_metrics)

        results_df = pd.DataFrame.from_records(results_df_rows)
        results_df.to_csv(f"results/{dataset_name}.csv")

Using cuda:0


wandb: Currently logged in as: justinhong (azizi-causal-perturb). Use `wandb login --relogin` to force relogin


'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'


Epoch 0: loss=13.03, score=8, shd=8.0, gamma=0.00
Node  0: | | 1 4 9 8 7 | 2 6 5 3 0
Node  1: | | 8 0 7 4 9 | 5 2 3 6 1
Node  2: | | 3 | 7 6 8 4 0 9 1 5 2
Node  3: | | 2 | 6 7 1 4 9 0 5 8 3
Node  4: | | 0 1 8 9 7 6 | 5 2 3 4
Node  5: | | | 4 7 2 3 9 1 6 8 0 5
Node  6: | | 4 | 3 1 0 2 9 7 8 5 6
Node  7: | | 1 8 0 4 9 | 3 5 2 6 7
Node  8: | | 1 0 4 7 9 | 2 6 5 3 8
Node  9: | | 0 1 4 8 7 | 6 5 2 3 9
Thresholds t:
	t >0.5: is_dag=True, shd=8.0
	t >0.3: is_dag=True, shd=8.0
	t >0.1: is_dag=False, shd=26.0


Epoch 100: loss=7.91, score=9, shd=9.0, gamma=0.00
Node  0: 1 | 4 9 | 6 | 8 5 2 7 3 0
Node  1: | 8 7 0 | 4 6 | 9 3 5 2 1
Node  2: 3 | | | 7 6 5 9 4 8 0 1 2
Node  3: 2 | | | 0 9 6 5 7 1 8 4 3
Node  4: 0 6 | 1 | | 8 3 7 9 2 5 4
Node  5: | | | 2 3 7 9 1 4 8 6 0 5
Node  6: 4 0 1 | | | 8 9 5 7 2 3 6
Node  7: 1 | | | 0 4 6 3 9 2 5 8 7
Node  8: 1 | | | 4 7 9 6 0 5 2 3 8
Node  9: 0 | | | 4 5 3 1 2 7 6 8 9
Thresholds t:
	t >0.5: is_dag=False, shd=7.0
	t >0.3: is_dag=False, shd=9.0
	t >0.1: is_dag

alpha,██████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
dag,▁▁▁▁▁▁▁▁▁▁▂▆▃▃▄██▇▇██▇▇▆▆▆▆▆▅▅
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_loss,▇▁▁▁▁▁▁▁▁▁█▄▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂
gamma,▁▁▁▁▁▁▁▁▁▁▁▅██████████████████
is_prescreen,██████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l1,▃█▇▆▆▆▆▅▅▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l2,▁▂▃▃▃▄▄▄▄▄▁▁▁▂▂▂▃▃▄▄▅▅▆▆▇▇▇▇██
n_edges_pred,▁█▇▆▆▆▆▆▆▆▁▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
nll,▇▁▁▁▁▁▁▁▁▁█▄▄▄▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
precision,▁▄▅▅▅▅▅▅▅▅▁▇██████████████████


'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'


Epoch 0: loss=13.06, score=10, shd=10.0, gamma=0.00
Node  0: | | 6 4 1 2 | 9 8 3 7 5 0
Node  1: | | 4 9 8 | 0 5 7 3 6 2 1
Node  2: | | 4 9 | 0 8 5 3 7 6 1 2
Node  3: | | 5 | 0 1 9 2 7 8 6 4 3
Node  4: | | 1 2 0 9 | 8 6 5 7 3 4
Node  5: | | 3 8 | 9 1 2 0 6 4 7 5
Node  6: | | 0 | 4 5 9 2 3 1 8 7 6
Node  7: | | | 4 1 3 6 9 2 5 0 8 7
Node  8: | | 9 5 1 4 | 2 0 3 6 7 8
Node  9: | | 8 1 4 2 5 0 | 6 7 3 9
Thresholds t:
	t >0.5: is_dag=True, shd=10.0
	t >0.3: is_dag=True, shd=10.0
	t >0.1: is_dag=False, shd=19.0


Epoch 100: loss=7.87, score=16, shd=16.0, gamma=0.00
Node  0: 6 4 | | | 2 8 1 3 7 5 9 0
Node  1: 4 5 2 3 8 | | | 9 7 0 6 1
Node  2: 4 1 9 | | | 8 5 3 6 7 0 2
Node  3: 5 8 1 | | | 2 6 0 4 9 7 3
Node  4: 1 2 | | 9 0 6 | 8 5 7 3 4
Node  5: 3 8 1 | | | 4 2 7 9 6 0 5
Node  6: 0 4 | | | 7 8 5 2 3 9 1 6
Node  7: | | | 0 6 1 5 9 4 2 3 8 7
Node  8: 9 5 | 3 | 1 | 2 4 0 7 6 8
Node  9: 4 2 8 | | | 1 5 7 0 6 3 9
Thresholds t:
	t >0.5: is_dag=False, shd=15.0
	t >0.3: is_dag=False, shd=16.0
	t >0.1

alpha,█████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
dag,▁▁▁▁▁▁▁▁▁▁█▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch_loss,█▁▁▁▁▁▁▁▁█▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂
gamma,▁▁▁▁▁▁▁▁▁▁▅██████████████████
is_prescreen,█████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l1,▂█▇▆▆▆▆▆▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
l2,▁▂▃▃▃▃▃▃▄▁▁▂▃▃▄▄▅▅▅▆▆▆▇▇▇▇███
n_edges_pred,▁████████▁▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
nll,█▁▁▁▁▁▁▁▁█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
precision,▁▄▄▄▄▄▄▄▄▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [11]:
def run_sdci(): 
    wandb.init()
    s1_alpha = wandb.config.s1_alpha
    s2_alpha = wandb.config.s2_alpha
    s1_beta = wandb.config.s1_beta
    s2_beta = wandb.config.s2_beta
    max_gamma = wandb.config.max_gamma

    model = SDCI(model_variance_flavor="nn")
    model.train(
        train_dataset,
        val_dataset=val_dataset,
        B_true=B_true,
        device=device,
        verbose=False,
        log_wandb=True,
        wandb_project=f"{dataset_name}",
        wandb_name=f"SDCI_{dataset_i}",
        stage1_kwargs={"n_epochs": 2000, "alpha": s1_alpha, "beta": s1_beta},
        stage2_kwargs={"n_epochs": 2000, "alpha": s2_alpha, "beta": s2_beta, "max_gamma": max_gamma}
    )

    y_test = B_true.astype(bool)
    y_score = model.get_adjacency_matrix(threshold = True)
    final_shd = shd_metric(y_score, y_test)

    wandb.log(dict(final_shd=final_shd))
    wandb.finish()

In [12]:
sweep_configuration = {
    "method" : "bayes",
    "name": "sdci_new_sim_sweep",
    "metric": {
        "goal": "minimize",
        "name": "final_shd",
    },
    "parameters": {
        "s1_alpha": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "s2_alpha": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "s1_beta": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "s2_beta": {"max": 1e-1, "min": 1e-5, "distribution": "log_uniform_values"},
        "max_gamma": {"max": 1000, "min": 100, "distribution": "log_uniform_values"},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 11,
    }
}

In [13]:
# wandb.login()

# sweep_id = wandb.sweep(sweep=sweep_configuration, project=f"{dataset_name}")
# wandb.agent(sweep_id, function=run_sdci)

wandb: Currently logged in as: justinhong (azizi-causal-perturb). Use `wandb login --relogin` to force relogin


Create sweep with ID: g8n58o0l
Sweep URL: https://wandb.ai/azizi-causal-perturb/data_p10_e10_n10000_linear_struct/sweeps/g8n58o0l


wandb: Agent Starting Run: 2ectxgnl with config:
wandb: 	max_gamma: 787.1452152785826
wandb: 	s1_alpha: 0.0002879899589071379
wandb: 	s1_beta: 0.007721089143342407
wandb: 	s2_alpha: 4.401203967905305e-05
wandb: 	s2_beta: 1.0618140779016947e-05


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'


...

'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'


Epoch 0: loss=13.52, score=8, shd=8.0, gamma=0.00
Epoch 100: loss=7.54, score=19, shd=19.0, gamma=0.00
Epoch 200: loss=7.52, score=27, shd=27.0, gamma=0.00
Epoch 300: loss=7.51, score=32, shd=32.0, gamma=0.00
Epoch 400: loss=7.48, score=40, shd=40.0, gamma=0.00
Epoch 500: loss=7.48, score=46, shd=46.0, gamma=0.00
Epoch 600: loss=7.50, score=56, shd=56.0, gamma=0.00
Epoch 700: loss=7.48, score=59, shd=59.0, gamma=0.00
Epoch 800: loss=7.46, score=62, shd=62.0, gamma=0.00
Epoch 900: loss=7.36, score=67, shd=67.0, gamma=0.00
Epoch 1000: loss=7.33, score=67, shd=67.0, gamma=0.00
Epoch 1100: loss=7.34, score=68, shd=68.0, gamma=0.00
Epoch 1200: loss=7.31, score=66, shd=66.0, gamma=0.00
Epoch 1300: loss=7.30, score=67, shd=67.0, gamma=0.00
Epoch 1400: loss=7.30, score=68, shd=68.0, gamma=0.00
Epoch 1500: loss=7.29, score=68, shd=68.0, gamma=0.00
Epoch 1600: loss=7.28, score=68, shd=68.0, gamma=0.00
Epoch 1700: loss=7.31, score=72, shd=72.0, gamma=0.00
Epoch 1800: loss=7.29, score=72, shd=72.0

alpha,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████
dag,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂█▃▃▃
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
epoch_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▆▆▆▆
final_shd,▁
gamma,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄███
is_prescreen,███████████████████▁▁▁▁▁
l1,▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▂████
l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂█▅▅▅
n_edges_pred,▁▃▄▅▅▆▇▇▇███▇██████▁▂▂▂▂
nll,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▃▄▄▄


wandb: Agent Starting Run: 9ig71dr7 with config:
wandb: 	max_gamma: 922.3369251758552
wandb: 	s1_alpha: 4.866390305914235e-05
wandb: 	s1_beta: 0.02296600438530688
wandb: 	s2_alpha: 0.0010348426877504018
wandb: 	s2_beta: 4.858241874749474e-05


wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: Ctrl + C detected. Stopping sweep.
